In [1]:
import sys
sys.path.insert(0,"")

from StreetGraph import StreetGraph
from gym_graphenv.envs.Graphworld import GraphEnv


#from gym_example.envs.Gridworld_Gym import Gridworld_v1


#graph_meinheim=StreetGraph('meinheim')
#graph_meinheim_trips = graph_meinheim.trips

In [2]:
env=GraphEnv()

In [31]:
env.availableActions()

[[{'type': 'wait'}], [{'type': 'ownRide'}]]

In [72]:
env.step(action=2)

[[{'type': 'wait'}], [{'type': 'ownRide'}]]


(5, -1, False, {})

In [3]:
env.graph.get_nodeid_by_index(env.position)

290333444

In [4]:
graph_meinheim=StreetGraph('meinheim')
graph_meinheim_trips = StreetGraph('meinheim').trips
print(graph_meinheim_trips)

      tripid  pickup_node  dropoff_node  pickup_day  pickup_hour  \
0          0    307664242     290333858           1           23   
1          1    280324485     306225115           1            1   
2          2    306221807     291556214           1           20   
3          3    306221765     307662889           1            2   
4          4    309441562     354863048           1           17   
...      ...          ...           ...         ...          ...   
3995    3995    306225115     309450366           1            3   
3996    3996    307662889     306219024           1           10   
3997    3997    280324491     290333400           1           17   
3998    3998    309450340     306219024           1            7   
3999    3999    307661801     309450229           1           12   

      pickup_minute     pickup_datetime  \
0                18 2022-01-01 23:18:00   
1                42 2022-01-01 01:42:00   
2                10 2022-01-01 20:10:00   
3          

In [2]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(30):
        print(env.action_space)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("sum_reward: ",sum_reward, " time: ",env.time,  "deadline time: ", env.deadline,"pickup time: ", env.pickup_time )
            break

        print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

env=GraphEnv()
for i in range(1):
    run_one_episode (env)

Discrete(3)
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 21, 8, 21), 'target_node': 307664242, 'route': [290333444, 290333443, 307664257, 307664242]}]
action == wait 
sum_reward:  -1  time:  2022-01-01 21:10:00 deadline time:  2022-01-02 00:05:00 pickup time:  2022-01-01 21:05:00
Discrete(3)
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 21, 11, 45), 'target_node': 290333443, 'route': [290333444, 290333443]}]
action ==  ownRide 
sum_reward:  49  time:  2022-01-01 21:19:19.600000 deadline time:  2022-01-02 00:05:00 pickup time:  2022-01-01 21:05:00


In [3]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [4]:
ray.init()

2022-04-02 12:54:28,300	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:32828',
 'object_store_address': '/tmp/ray/session_2022-04-02_12-54-26_331985_41617/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-02_12-54-26_331985_41617/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2022-04-02_12-54-26_331985_41617',
 'metrics_export_port': 58148,
 'gcs_address': '127.0.0.1:56855',
 'node_id': 'e2cad86cda603008231414b3af164f884f910642b11086b2f8aa9ea7'}

In [5]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10
trainer_config["framework"] = "torch"

In [6]:
trainer = PPOTrainer(trainer_config,GraphEnv )

2022-04-02 12:54:35,868	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-04-02 12:54:35,870	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=41749) 2022-04-02 12:54:41,988	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-02 12:54:42,849	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [7]:
checkpoint_root = "tmp/ppo/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [8]:
trainer.train()

(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 13, 6, 47), 'target_node': 306222385, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385]}, {'departure_time': datetime.datetime(2022, 1, 1, 13, 10, 40), 'target_node': 307664257, 'route': [290333444, 290333443, 307664257]}]
(RolloutWorker pid=41749) action ==  3  New Position 44
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 13, 13, 12), 'target_node': 307664334, 'route': [307664257, 307664242, 307664334]}, {'departure_time': datetime.datetime(2022, 1, 1, 13, 11, 22), 'target_node': 2586413037, 'route': [307664257, 290333443, 290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900, 306221960, 306222676, 306221941, 2586413037]}]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownR

{'episode_reward_max': 50.0,
 'episode_reward_min': 25.0,
 'episode_reward_mean': 43.892857142857146,
 'episode_len_mean': 7.107142857142857,
 'episode_media': {},
 'episodes_this_iter': 56,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [30.0,
   36.0,
   42.0,
   44.0,
   40.0,
   46.0,
   49.0,
   49.0,
   48.0,
   43.0,
   49.0,
   45.0,
   32.0,
   47.0,
   46.0,
   43.0,
   50.0,
   47.0,
   50.0,
   48.0,
   49.0,
   49.0,
   41.0,
   46.0,
   30.0,
   42.0,
   44.0,
   49.0,
   48.0,
   50.0,
   39.0,
   46.0,
   44.0,
   48.0,
   42.0,
   50.0,
   50.0,
   33.0,
   34.0,
   32.0,
   48.0,
   48.0,
   34.0,
   45.0,
   50.0,
   43.0,
   45.0,
   48.0,
   42.0,
   49.0,
   46.0,
   48.0,
   25.0,
   47.0,
   43.0,
   47.0],
  'episode_lengths': [21,
   15,
   9,
   7,
   11,
   5,
   2,
   2,
   3,
   8,
   2,
   6,
   19,
   4,
   5,
   8,
   1,
   4,
   1,
   3,
   2,
   2,
   10,
   5,
  

NameError: name 'result' is not defined

In [9]:
results = []
episode_data = []
episode_json = []
n_iter=5

for n in range(n_iter):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) action == wait 
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) action == wait 
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) action == wait 
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.dateti

2022-04-02 13:06:39,944	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


(RolloutWorker pid=41749) action ==  ownRide 
  1: Min/Mean/Max reward:  18.0000/ 44.8300/ 83.0000, len mean:   6.6700. Checkpoint saved to tmp/ppo/graphworld/checkpoint_000002/checkpoint-2
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=41749) action ==  ownRide 
(RolloutWorker pid=41749) [[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 1, 43, 3), 'target_node': 307664334, 'route': [290333444, 290333443, 307664257, 307664242, 307664334]}, {'departure_time': datetime.datetime(2022, 1, 1, 1, 42, 43), 'target_node': 306222676, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900, 306221960, 306222676]}, {'departure_time': datetime.datetime(2022, 1, 1, 1, 40, 23), 'target_node': 306222676, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900, 306221960, 306222676]}, {'departu

In [11]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

2022-04-02 13:49:45,957	INFO trainable.py:472 -- Restored on 127.0.0.1 from checkpoint: tmp/ppo/graphworld/checkpoint_000006/checkpoint-6
2022-04-02 13:49:45,959	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 6, '_timesteps_total': 2400, '_time_total': 2637.487790107727, '_episodes_total': 595}


In [13]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 3, 46), 'target_node': 306226719, 'route': [280430536, 307661801, 307662284, 309450366, 307662242, 306222128, 309450229, 306226719]}]
cumulative reward 50
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 6, 19, 42), 'target_node': 306221765, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900, 306225115, 306225011, 306221653, 306225000, 306221765]}, {'departure_time': datetime.datetime(2022, 1, 1, 6, 22, 56), 'target_node': 307664257, 'route': [290333444, 290333443, 307664257]}]
action ==  ownRide 
cumulative reward 50
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 11, 10, 21), 'target_node': 306221802, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900, 306225115, 30622501

In [ ]:
ray.shutdown()